In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import math
from math import cos, acos, sin , asin, pi, radians, sqrt, degrees

## Data Processing

In [2]:
##  Data Processing

def dataProcess(inputFile):
  tempData = []
  with open(inputFile) as f:
    for line in f:
      tempString = line
      dateString = tempString[0:9]
      dateString = dateString.replace(" ", ":")
      finalString = dateString + tempString[9:len(tempString)]
      finalString = finalString.strip("\n")
      finalList = finalString.split(",")
      tempData.append(finalList)

  data = pd.DataFrame(tempData, columns = ["Date", "Time", "RA", "Dec", "Uncertainty RA", "Uncertainty DEC"])
  data['RA'] = raToDecimal(data['RA'])
  data['Dec'] = decToDecimal(data['Dec'])
  data['JD'] = obtainJDDates(data['Date'], data['Time'])
  return data


def raToDecimal(data):
  decimalRA = []
  for i in range(len(data)):
    tempValues = np.array(data[i].split(":")).astype(float)
    tempDecimal = (tempValues[0] + (tempValues[1]/60) + (tempValues[2]/3600)) * 15
    tempDecimal = radians(tempDecimal)
    decimalRA.append(tempDecimal)
  return decimalRA


def decToDecimal(data):
  decimalDEC = []
  for i in range(len(data)):
    tempString = data[i]
    tempValues = np.array(tempString.split(":")).astype(float)
    if tempValues[0] >= 0:
      tempDecimal = tempValues[0] + (tempValues[1]/60) + (tempValues[2]/3600)
    else:
      tempDecimal = tempValues[0] - (tempValues[1]/60) - (tempValues[2]/3600)
    tempDecimal = radians(tempDecimal)
    decimalDEC.append(tempDecimal)
  return decimalDEC


def obtainJDDates(dateData, timeData):
  dataJD = []
  for i in range(len(dateData)):
    tempDateData = np.array(dateData[i].split(":")).astype(int).tolist()
    tempTimeData = np.array(timeData[i].split(":")).tolist()
    
    H = int(tempTimeData[0])
    M = int(tempTimeData[1])
    S = int(float(tempTimeData[2]))

    tempTime = pd.Timestamp(year = tempDateData[0], month = tempDateData[1], day = tempDateData[2], hour = H, minute = M, second = S)
    tempTime = tempTime.to_julian_date()
    dataJD.append(float(tempTime))

  return np.array(dataJD)

def arcSecToRadians(arcSec):
  output = arcSec *  math.pi/(180 * 3600)
  return output

## MOG

### Functions for OD

In [3]:

def obtainRowHats(raData, decData):

  iHat = np.array([1, 0, 0])
  jHat = np.array([0, 1, 0])
  kHat = np.array([0, 0, 1])

  rowHats = []

  for i in range(len(raData)):
    tempPVec = (cos(raData[i]) * cos(decData[i])) * iHat + (sin(raData[i]) * cos(decData[i])) * jHat + sin(decData[i]) * kHat
    rowHats.append(tempPVec)
  
  return rowHats

def obtainDValues(rowHats, sunVectors):
  
  dValues = []

  for i in range(3):

    d1 = np.dot(np.cross(sunVectors[i], rowHats[1]), rowHats[2])
    d2 = np.dot(np.cross(rowHats[0], sunVectors[i]), rowHats[2])
    d3 = np.dot(np.cross(rowHats[1], sunVectors[i]), rowHats[0])

    dValues.append(d1)
    dValues.append(d2)
    dValues.append(d3)

  d0 = np.dot(np.cross(rowHats[1], rowHats[2]), rowHats[0])

  return d0, dValues[0], dValues[1], dValues[2], dValues[3], dValues[4], dValues[5], dValues[6], dValues[7], dValues[8]

def obtainMagnitude(vector):
  vectorMag = math.sqrt(vector[0]**2 + vector[1]**2 + vector[2]**2)
  return vectorMag

#Scalar Equation of Lagrange

def SEL(taus, Sun2, rhohat, Dvalues, choose = False):
  taus = np.array(taus)
  Sun2 = np.array(Sun2)
  rhohat = np.array(rhohat)

  mu = 1
  t = taus[2]
  a1 = taus[1]/t 
  b1 = (a1/6)*(t**2 - taus[1]**2)
  a3 = (-taus[0])/t
  b3 = ((a3)/6) * (t**2 - taus[0]**2)

  A = (a1 * Dvalues[1] - Dvalues[2] + a3 * Dvalues[3])/(- Dvalues[0])
  B = (b1 * Dvalues[1] + b3 * Dvalues[3])/( - Dvalues[0])

  F = obtainMagnitude(Sun2)**2
  E = -2 * (np.dot(rhohat,Sun2))
  
  aCoeff = - (A**2 + A * E + F)
  bCoeff = - mu * (2 * A * B + B * E)
  cCoeff = - (mu ** 2) * (B ** 2)

  coEffArray = np.asarray([1, 0, aCoeff, 0, 0, bCoeff, 0, 0, cCoeff])

  roots = np.roots(coEffArray)

  #Obtain only real positive numbers
  for i in range(len(roots)):
    if ((np.isreal(roots[i]) == False)):
      roots[i] = 0

  roots = roots[roots != 0]
  roots = roots[roots >= 0]
  roots = roots.astype(float)

  #Obtain Rhos

  rows = []
  for i in range(len(roots)):
    rows.append(A + (B/((roots[i])**3)))
    
  rows = np.array(rows)

  #Filter Roots from negative Rhos / negative ranges

  for i in range(len(rows)):
    if (rows[i] < 0):
      roots[i] = 0

  roots = roots[roots != 0]
  rows = rows[rows >= 0]

  if choose == True:

    print("r2Mags: ", roots.tolist())
    print("Rows: ", rows.tolist())


    indexDesired = int(input(("Which roots/rows would you like to utilize? (0 indexed): ")))
    print("\n\n")
  
  else:

    indexDesired = 0

  return roots[indexDesired], rows[indexDesired]

# F and G Calculations

def getE(M, e, tolerance):
    Eguess = M
    Mguess = Eguess - e*sin(Eguess)
    while abs(Mguess - M) > tolerance:
        Mguess = Eguess - e*sin(Eguess)
        Eguess = Eguess - (M - (Eguess - (e*sin(Eguess)))) / (e*cos(Eguess) - 1)
    return Eguess

def fg(tau1, r2, flag, r2dot = None):
  mu = 1

  #Taylor Series: 1st Order

  if (flag == "1st"):

    #For First Series, the r2 paramter is the r2 Mag, not the vector
    rPositionMag = r2

    f = 1 - (mu/ (2 * (rPositionMag**3))) * tau1 ** 2
    g = tau1 - (mu/ (6 * (rPositionMag**3))) * tau1 ** 3

  if (flag == "3rd"):
    rPositionMag = obtainMagnitude(r2)
    rVelocityMag  = obtainMagnitude(r2dot)

    u = mu/(rPositionMag**3) 
    z = (np.dot(r2, r2dot))/(rPositionMag ** 2)
    q = ((np.dot(r2dot, r2dot))/(rPositionMag ** 2)) - u

    f = 1 - .5 * (u * tau1 ** 2) + .5 * (u * z * tau1 ** 3)
    g = tau1 - (mu/(6 * (rPositionMag**3))) * tau1 ** 3

  if (flag == "4th"):
    rPositionMag = obtainMagnitude(r2)
    rVelocityMag  = obtainMagnitude(r2dot)

    u = mu/(rPositionMag**3) 
    z = (np.dot(r2, r2dot))/(rPositionMag ** 2)
    q = ((np.dot(r2dot, r2dot))/(rPositionMag ** 2)) - u
  
    f = 1 - .5 * (u * tau1 ** 2) + .5 * (u * z * tau1 ** 3) + ((1/24) * (3 * u * q - 15 * u * z ** 2 + u**2) * tau1 ** 4)
    g = tau1 - ((1/6)* u * tau1 ** 3) + (1/4)*u*z*tau1**4
  
  if (flag == "Function"):

    rPositionMag = obtainMagnitude(r2)
    rVelocityMag  = obtainMagnitude(r2dot)

    #Calculate A
    a = ((2/rPositionMag) - rVelocityMag * rVelocityMag) ** - 1

    #Calculate N
    n = 1/math.sqrt((a**3))

    x = n * tau1

    while True:

      fx = x - (1 - (rPositionMag/a)) * sin(x) + (np.dot(r2, r2dot)/(n * (a**2))) * (1 - cos(x)) - n * tau1

      fxPrime = 1 - (1 - (rPositionMag/a)) * cos(x) + (np.dot(r2, r2dot)/(n * (a**2))) * sin(x)

      xNew = x - (fx/fxPrime)

      if abs(xNew - x) < 1E-12:

        x = xNew

        break

      x = xNew 

    deltaE = x

    f = 1 - (a/rPositionMag) * (1 - cos(deltaE))
    g = tau1 + (1/n) * (sin(deltaE) - deltaE)

  return round(f, 20), round(g, 20)

def obtainTValues(jdData):
  k = 0.0172020989484
  t3 = k * (jdData[2] - jdData[1])
  t1 = k * (jdData[0] - jdData[1])
  t = k * (jdData[2] - jdData[0])
  return t, t1, t3

def obtainCValues(f1, f3, g1, g3):
  c1 = (g3)/(f1 * g3 - g1 * f3)
  c2 = -1
  c3 = (-g1)/(f1* g3 - g1*f3)

  return c1, c2, c3

def obtainRowMag(c1, c2, c3, d0, d11, d21, d31, d12, d22, d32, d13, d23, d33):
  p1 = (c1 * d11 + c2 * d12 + c3 * d13) / (c1 * d0)

  p2 = (c1 * d21 + c2 * d22 + c3 * d23) / (c2 * d0)

  p3 = (c1 * d31 + c2 * d32 + c3 * d33) / (c3 * d0)

  return p1, p2, p3

def obtainSunVectors(sunXData, sunYData, sunZData):
  sunVectors = []
  for i in range(len(sunXData)):
    sunVectors.append([sunXData[i], sunYData[i], sunZData[i]])
  return sunVectors[0], sunVectors[1], sunVectors[2]

def obtainAsteroidVectors(rowMagData, rowHatData, sunVectors):
  rVectors = []
  for i in range(len(rowMagData)):
    tempRVector = (rowMagData[i] * np.array(rowHatData[i])) - np.array(sunVectors[i])
    rVectors.append(np.array(tempRVector))
  return rVectors

def calcNewTimeValues(timeValues, rowMags):
  newTime = []

  for i in range(3): 
    tempTime = (timeValues[i]) - (rowMags[i])/173.144632674240  #Accesses the prior time values
    newTime.append(float(tempTime))

  return newTime

#Converging to find final Asteroid Vectors

def obtainConvAsteroidVectors(initialAsteroidPosVector, initialAsteroidVelVector, dValues, timeValues, rowMags, rowHats, sunVectors):

  #Variable Definitions

  rowHats = rowHats

  d0 = dValues[0]
  d11 = dValues[1]
  d21 = dValues[2]
  d31 = dValues[3]
  d12 = dValues[4]
  d22 = dValues[5]
  d32 = dValues[6]
  d13 = dValues[7]
  d23 = dValues[8]
  d33 = dValues[9]
  
  sunVector1 = sunVectors[0]
  sunVector2 = sunVectors[1]
  sunVector3 = sunVectors[2]

  row2MagData = [rowMags[1]]

  #Define New Vectors

  newAsteroidPosVector = initialAsteroidPosVector.tolist()

  newAsteroidVelVector = initialAsteroidVelVector.tolist()

  #Iterations 

  count = 1

 # print("Iteration Convergance for Asteroid Vector:\n")
 # fgFlag = str(input("\nWhat would you like to use for F & G values? (Options: 1st, 3rd, 4th, and Function): "))

  while (True):

    #Time Calculations

    newTimes = calcNewTimeValues(timeValues, rowMags)
    
    tau, tau1, tau3 = obtainTValues(newTimes)

    f1, g1 = fg(tau1, newAsteroidPosVector, "Function", r2dot = newAsteroidVelVector)

    f3, g3 = fg(tau3, newAsteroidPosVector, "Function", r2dot = newAsteroidVelVector)

    c1, c2, c3 = obtainCValues(f1, f3, g1, g3)

    row1Mag, row2Mag, row3Mag = obtainRowMag(c1, c2, c3, d0, d11, d21, d31, d12, d22, d32, d13, d23, d33)

    rowMags = [row1Mag, row2Mag, row3Mag]

    row2MagData.append(row2Mag)

    d1 = (-f3)/(f1 * g3 - f3 * g1)

    d3 = f1/(f1 * g3 - f3 * g1)

    rVectors = obtainAsteroidVectors([row1Mag, row2Mag, row3Mag], rowHats, [sunVector1, sunVector2, sunVector3])

    #Define New Asteroid Vectors

    newAsteroidPosVector = rVectors[1]

    newAsteroidVelVector = d1 * rVectors[0] + d3 * rVectors[2]

    count += 1

 #   print("Row2Mag Difference (" + str(count) + "):", row2MagData[len(row2MagData) - 1] - row2MagData[len(row2MagData) - 2])

    if (abs(row2MagData[len(row2MagData) - 1] - row2MagData[len(row2MagData) - 2]) < 1E-12 or count > 200):

      break

  return newAsteroidPosVector, newAsteroidVelVector, row2MagData[len(row2MagData) - 1]

#Baby OD: Obtaining Orbital Elements

def findQuadrant(sine, cosine):
    if cosine > 0 and sine > 0: #1
        return asin(sine)

    if cosine < 0 and sine > 0: #2
        return acos(cosine)

    if cosine < 0 and sine < 0: #3
        return pi - asin(sine)

    if cosine > 0 and sine < 0: #4
        return 2*pi + asin(sine)

def obtainEcliptic(vector):
  transMatrix = [[1, 0, 0],
                 [0, cos(radians(23.4379)), sin(radians(23.4379))],
                 [0, -sin(radians(23.4379)), cos(radians(23.4379))]]

  transMatrix = np.array(transMatrix)
  vector = np.array(vector)

  ecliptic = transMatrix @ vector
  
  return ecliptic

def obtainOrbitalElements(rPosition, rVelocity, time):
  rPosition = obtainEcliptic(rPosition)
  rVelocity = obtainEcliptic(rVelocity)

  rPositionMag = obtainMagnitude(rPosition)
  rVelocityMag = obtainMagnitude(rVelocity)


  #Calculate A
  a = ((2/rPositionMag) - rVelocityMag * rVelocityMag) ** - 1

  #Calculate N
  n = 0.0172020989484/math.sqrt((a**3))

  #Calculate Eccentricity
  e = math.sqrt(1 - ((obtainMagnitude(np.cross(rPosition, rVelocity))**2)/a))

  #Caclulate Inclination
  h = np.cross(rPosition, rVelocity)
  zComp = h[2]
  I = math.acos(zComp/obtainMagnitude(h))

  #Calculate Ascending Node
  hMag = obtainMagnitude(h)
  ascNodeSin = h[0]/(hMag * sin(I))
  ascNodeCos = (-h[1])/(hMag * sin(I))
  ascNode = findQuadrant(ascNodeSin, ascNodeCos)

  #Calculate Perihelion

  #5.1 
  sinFW = rPosition[2]/(rPositionMag * sin(I))
  cosFW = (1/cos(ascNode)) * ((rPosition[0]/rPositionMag) + cos(I) * sinFW * sin(ascNode))
  FW = findQuadrant(sinFW, cosFW)

  #5.2
  cosF =  (1/e) * (((a * (1-e**2))/rPositionMag) - 1)
  posVelCross = np.dot(rPosition, rVelocity)
  sinF = (posVelCross/(e*rPositionMag))*sqrt(a*(1-e**2))

  f = findQuadrant(sinF, cosF)
  w = (FW - f)

  # Calculate Mean Anomaly

  E = acos((1/e) * (1 - (rPositionMag/a)))

  if ((f >= 0 and f <= pi) and (E > pi and E < 2 * pi)):

      E = 2 * pi - E

  elif ((f >= pi and f <= 2 * pi) and (E > 0 and E < pi)):

      E = 2 * pi - E

  M = E - e * sin(E)

  M0 = E - e * sin(E) + n * (2459784.75 - time)

  while (M0 > 2 * pi):

    M0 -= 2 * pi

  while (M < 0):

    M0 += 2 * pi

  #Solve for T

  T = ((-M0)/n) + time

  #Solve for Period

  P = ((2*pi)/n)/365.256363004

  return a, e, degrees(I), degrees(ascNode), (degrees(w) + 360)%360, degrees(M), degrees(M0), degrees(E), n, T, P

### Main OD

In [4]:
#Main OD Code

def odCode(sunVectors, raData, decData, jdTimes):

  k = 0.0172020989484

  rowHats = obtainRowHats(raData, decData)

  sunVector1 = sunVectors[0] 
  sunVector2 = sunVectors[1]
  sunVector3 = sunVectors[2]

  d0, d11, d21, d31, d12, d22, d32, d13, d23, d33 = obtainDValues(rowHats, [sunVector1, sunVector2, sunVector3])

  timeValuesArray = jdTimes

  tau, tau1, tau3 = obtainTValues(jdTimes)

  r2Mag, thisISStupid = SEL([tau1, tau3, tau], sunVector2, rowHats[1], [d0, d21, d22, d23])

  f1, g1 = fg(tau1, r2Mag, "1st")
  f3, g3 = fg(tau3, r2Mag, "1st")
  
  c1, c2, c3 = obtainCValues(f1, f3, g1, g3)

  row1Mag, row2Mag, row3Mag = obtainRowMag(c1, c2, c3, d0, d11, d21, d31, d12, d22, d32, d13, d23, d33)

  d1 = (-f3)/(f1 * g3 - f3 * g1)
  d3 = f1/(f1 * g3 - f3 * g1)

  rVectors = obtainAsteroidVectors([row1Mag, row2Mag, row3Mag], rowHats, [sunVector1, sunVector2, sunVector3])

  r2VelocityVector = d1 * rVectors[0] + d3 * rVectors[2]

  finalAsteroidPosVector, finalAsteroidVelVector, row2MagFinal = obtainConvAsteroidVectors(rVectors[1], r2VelocityVector, [d0, d11, d21, d31, d12, d22, d32, d13, d23, d33], timeValuesArray, [row1Mag, row2Mag, row3Mag], rowHats, [sunVector1, sunVector2, sunVector3])

  finalAsteroidPosVectorEcl = obtainEcliptic(finalAsteroidPosVector)
  finalAsteroidVelVectorEcl = obtainEcliptic(finalAsteroidVelVector)

  a, e, I, ascNode, w, M, M0, E, n, T, P = obtainOrbitalElements(finalAsteroidPosVector, finalAsteroidVelVector, jdTimes[1])

  return finalAsteroidPosVector, finalAsteroidVelVector, a, e, I, ascNode, w, M, M0, E, n * (180/pi), T, P

# Monte Carlo

In [5]:
#Permutation Data Frame Defining (Input)
dataObs = dataProcess('/content/txt')

originalInputData =  [dataObs['RA'][0], dataObs['Dec'][0], dataObs['Uncertainty RA'][0], dataObs['Uncertainty DEC'][0], dataObs['RA'][1], dataObs['Dec'][1], dataObs['Uncertainty RA'][1], dataObs['Uncertainty DEC'][1], dataObs['RA'][2], dataObs['Dec'][2], dataObs['Uncertainty RA'][2], dataObs['Uncertainty DEC'][2], dataObs['JD'][0], dataObs['JD'][1], dataObs['JD'][2]]

monteCarloDataInput = { 'RA (1)' : [],
                   'DEC (1)': [],
                   'Uncert RA (1)' : [],
                   'Uncert DEC (1)' : [],
                   'RA (2)': [],
                   'DEC (2)': [],
                   'Uncert RA (2)' : [],
                   'Uncert DEC (2)' : [],
                   'RA (3)': [],
                   'DEC (3)': [],
                   'Uncert RA (3)' : [],
                   'Uncert DEC (3)' : [],
                   'JD (1)': [],
                   'JD (2)': [],
                   'JD (3)': []}

monteCarloDataInput = pd.DataFrame(monteCarloDataInput)

In [6]:
#Permutation Output Data Frame
monteCarloDataOutput = {'Asteroid Pos Vector' : [],
                        'Asteroid Vel Vector' : [],
                        'Semimajor Axis (a)' : [],
                        'Eccentricity (e)' : [],
                        'Inclination (I)' : [],
                        'Longitude of Ascending Node (Ω)' : [],
                        'Argument of Periapsis (ω)' : [],
                        'Mean Anomoly (M)' : [],
                        'Mean Anomoly at 2022 July 24 6:00 UTC (M0)': [],
                        "Eccentric Anomoly (E)" : [],
                        "Mean Motion (n)" : [],
                        'Period (P)' : [] }
monteCarloDataOutput = pd.DataFrame(monteCarloDataOutput)

In [7]:
def obtainInputPermutations(ogInputData, size):

  newRA1 = np.random.normal(loc = ogInputData[0], scale = arcSecToRadians(float(ogInputData[2])), size = size)
  newDec1 = np.random.normal(loc = ogInputData[1], scale = arcSecToRadians(float(ogInputData[3])), size = size)

  newRA2 = np.random.normal(loc = ogInputData[4], scale = arcSecToRadians(float(ogInputData[6])), size = size)
  newDec2 = np.random.normal(loc = ogInputData[5], scale = arcSecToRadians(float(ogInputData[7])), size = size)

  newRA3 = np.random.normal(loc = ogInputData[8], scale = arcSecToRadians(float(ogInputData[10])), size = size)
  newDec3 = np.random.normal(loc = ogInputData[9], scale = arcSecToRadians(float(ogInputData[11])), size = size)

  return newRA1, newDec1, newRA2, newDec2, newRA3, newDec3

In [ ]:
newRA1, newDec1, newRA2, newDec2, newRA3, newDec3 = obtainInputPermutations(originalInputData, 1000000000)

monteCarloDataInput['RA (1)'] = newRA1.tolist()
monteCarloDataInput['DEC (1)'] = newDec1.tolist()

monteCarloDataInput['RA (2)'] = newRA2.tolist()
monteCarloDataInput['DEC (2)'] = newDec2.tolist()

monteCarloDataInput['RA (3)'] = newRA3.tolist()
monteCarloDataInput['DEC (3)'] = newDec3.tolist()

In [ ]:
#Data Preperation 

sunVectors = [[-1.258211413846450E-01, 9.256202763766458E-01, 4.012183811447692E-01], 
              [-3.250994505105637E-01, 8.837859903618946E-01, 3.830771048444394E-01], 
              [-4.482889143168946E-01, 8.368108443296683E-01, 3.627148461138572E-01]]

jdData = [originalInputData[12], originalInputData[13], originalInputData[14]]

print("OD Elements")

count = 0

for i in range(len(monteCarloDataInput)):

  raData = [monteCarloDataInput['RA (1)'][i], monteCarloDataInput['RA (2)'][i], monteCarloDataInput['RA (3)'][i]]
  decData = [monteCarloDataInput['DEC (1)'][i], monteCarloDataInput['DEC (2)'][i], monteCarloDataInput['DEC (3)'][i]]

  finalAsteroidPosVector, finalAsteroidVelVector, a, e, I, ascNode, w, M, M0, E, n, T, P = odCode(sunVectors, raData, decData, jdData)

  tempOutput = {'Asteroid Pos Vector' : finalAsteroidPosVector,
                        'Asteroid Vel Vector' : finalAsteroidVelVector,
                        'Semimajor Axis (a)' : a,
                        'Eccentricity (e)' : e,
                        'Inclination (I)' : I,
                        'Longitude of Ascending Node (Ω)' : ascNode,
                        'Argument of Periapsis (ω)' : w,
                        'Mean Anomoly (M)' : M,
                        'Mean Anomoly at 2022 July 24 6:00 UTC (M0)': M0,
                        "Eccentric Anomoly (E)" : E,
                        "Mean Motion (n)" : n,
                        'Period (P)' : P }

  count += 1

  print(count, " Iteration")

  monteCarloDataOutput = monteCarloDataOutput.append(tempOutput, ignore_index = True)

  if (count%1000 == 0):

    monteCarloDataOutput.to_csv('MonteCarloData.csv', index=False)

monteCarloDataOutput.to_csv('MonteCarloData.csv', index=False)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import statistics

In [ ]:
columns = ['Semimajor Axis (a)', 'Eccentricity (e)', 'Inclination (I)', 'Longitude of Ascending Node (Ω)', 'Argument of Periapsis (ω)', 'Mean Anomoly (M)', 'Mean Anomoly at 2022 July 24 6:00 UTC (M0)', 'Eccentric Anomoly (E)', 'Mean Motion (n)', 'Period (P)']

standardDeviation = []

for i in range(len(columns)):
  tempArray = monteCarloDataOutput[columns[i]].tolist()
  standardDeviationTemp = statistics.stdev(tempArray)
  standardDeviation.append([columns[i], standardDeviationTemp])

print("\n\n\n\nStandard Deviation")
print(standardDeviation)

In [ ]:
fig, axs = plt.subplots(2, 5, figsize = (40, 40))
sns.set_style('darkgrid')

sns.histplot(data = monteCarloDataOutput, x = "Semimajor Axis (a)", kde = True, ax = axs[0][0], color = "red")
sns.histplot(data = monteCarloDataOutput, x = "Eccentricity (e)", kde = True, ax = axs[0][1], color = "skyblue")
sns.histplot(data = monteCarloDataOutput, x = "Inclination (I)", kde = True, ax = axs[0][2], color = "gold")
sns.histplot(data = monteCarloDataOutput, x = "Longitude of Ascending Node (Ω)", kde = True, ax = axs[0][3], color = "blue")
sns.histplot(data = monteCarloDataOutput, x = "Argument of Periapsis (ω)", kde = True, ax = axs[0][4], color = "teal")
sns.histplot(data = monteCarloDataOutput, x = "Mean Anomoly (M)", kde = True, ax = axs[1][0], color = "purple")
sns.histplot(data = monteCarloDataOutput, x = "Mean Anomoly at 2022 July 24 6:00 UTC (M0)", kde = True, ax = axs[1][1], color = "olive")
sns.histplot(data = monteCarloDataOutput, x = "Eccentric Anomoly (E)", kde = True, ax = axs[1][2], color = "orange")
sns.histplot(data = monteCarloDataOutput, x = "Mean Motion (n)", kde = True, ax = axs[1][3], color = "green")
sns.histplot(data = monteCarloDataOutput, x = "Period (P)", kde = True, ax = axs[1][4], color = "navy")